## temp notebook for developing year over year comparison checks 

TODO: these same functions could easily be used to compare data after code changes as well. 

In [118]:
import numpy as np
import pandas as pd
import plotly.express as px
import os


In [121]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append("../../src")

from column_checks import get_dtypes
from filepaths import results_folder

# Experimenting with monthly data

In [3]:
get_dtypes()['report_date']

'datetime'

In [7]:
power_2019 = pd.read_csv("../../data/outputs/2019/gross_to_net_conversions_2019.csv", dtype=get_dtypes(), parse_dates=["report_date"])
power_2020 = pd.read_csv("../../data/outputs/2020/gross_to_net_conversions_2020.csv", dtype=get_dtypes(), parse_dates=["report_date"])

In [14]:
file_type_to_index_cols = {
    "gross_to_net_conversions_":["plant_id_eia","subplant_id","month"]
}


In [15]:
# Add months 
power_2020['month'] = power_2020.report_date.dt.month
power_2019['month'] = power_2019.report_date.dt.month

In [16]:
numeric_2019 = power_2019.set_index(file_type_to_index_cols["gross_to_net_conversions_"]).select_dtypes(include=[np.number]) 
numeric_2020 = power_2020.set_index(file_type_to_index_cols["gross_to_net_conversions_"]).select_dtypes(include=[np.number])

In [17]:
(numeric_2020 - numeric_2019)/numeric_2019

hours_in_month_subplant  gross_generation_mwh  \
plant_id_eia subplant_id month                                                  
3            0           1                          NaN                   NaN   
                         2                          NaN                   NaN   
                         3                          NaN                   NaN   
                         5                          NaN                   NaN   
                         7                          NaN                   NaN   
...                                                 ...                   ...   
61242        1           8                          0.0             -0.292672   
                         9                          NaN                   NaN   
                         10                         NaN                   NaN   
                         11                         NaN                   NaN   
                         12                         NaN                   NaN   

                                minimum_gross_generation_mwh  \
plant_id_eia subplant_id month                                 
3            0           1                               NaN   
                         2                               NaN   
                         3                               NaN   
                         5                               NaN   
                         7                               NaN   
...                                                      ...   
61242        1           8                               NaN   
                         9                               NaN   
                         10                              NaN   
                         11                              NaN   
                         12                              NaN   

                                maximum_gross_generation_mwh  capacity_mw  \
plant_id_eia subplant_id month                                              
3            0           1                               NaN          NaN   
                         2                               NaN          NaN   
                         3                               NaN          NaN   
                         5                               NaN          NaN   
                         7                               NaN          NaN   
...                                                      ...          ...   
61242        1           8                               0.0          0.0   
                         9                               NaN          NaN   
                         10                              NaN          NaN   
                         11                              NaN          NaN   
                         12                              NaN          NaN   

                                net_generation_mwh  monthly_subplant_ratio  \
plant_id_eia subplant_id month                                               
3            0           1                     NaN                     NaN   
                         2                     NaN                     NaN   
                         3                     NaN                     NaN   
                         5                     NaN                     NaN   
                         7                     NaN                     NaN   
...                                            ...                     ...   
61242        1           8               -0.120624                0.243236   
                         9                     NaN                     NaN   
                         10                    NaN                     NaN   
                         11                    NaN                     NaN   
                         12                    NaN                     NaN   

                                monthly_subplant_shift_mw  \
plant_id_eia subplant_id month                              
3     

In [18]:
numeric_2019.head()

hours_in_month_subplant  gross_generation_mwh  \
plant_id_eia subplant_id month                                                  
3            0           3                        743.0                2137.0   
                         5                        744.0                7660.0   
                         7                        744.0                4859.0   
                         9                        720.0               11190.0   
                         10                       744.0                3608.0   

                                minimum_gross_generation_mwh  \
plant_id_eia subplant_id month                                 
3            0           3                               0.0   
                         5                               0.0   
                         7                               0.0   
                         9                               0.0   
                         10                              0.0   

                                maximum_gross_generation_mwh  capacity_mw  \
plant_id_eia subplant_id month                                              
3            0           3                             60.00        153.1   
                         5                             76.14        153.1   
                         7                             27.00        153.1   
                         9                             27.00        153.1   
                         10                            86.00        153.1   

                                net_generation_mwh  monthly_subplant_ratio  \
plant_id_eia subplant_id month                                               
3            0           3                    68.0                0.031820   
                         5                   480.0                0.062663   
                         7                   163.9                0.033731   
                         9                  1233.0                0.110188   
                         10                   81.3                0.022533   

                                monthly_subplant_shift_mw  \
plant_id_eia subplant_id month                              
3            0           3                      -2.784657   
                         5                      -9.650538   
                         7                      -6.310618   
                         9                     -13.829167   
                         10                     -4.740188   

                                annual_subplant_ratio  \
plant_id_eia subplant_id month                          
3            0           3                   0.068792   
                         5                   0.068792   
                         7                   0.068792   
                         9                   0.068792   
                         10                  0.068792   

                                annual_subplant_shift_mw  ...  \
plant_id_eia subplant_id month                            ...   
3            0           3                      -7.42295  ...   
                         5                      -7.42295  ...   
                         7                      -7.42295  ...   
                         9                      -7.42295  ...   
                         10                     -7.42295  ...   

                                hours_in_month_plant  annual_plant_ratio  \
plant_id_eia subplant_id month                                             
3            0           3                   40246.0            0.961429   
                         5                   40246.0            0.961429   
                         7                   40246.0            0.961429   
                         9                   40246.0            0.961429   
                         10                  40246.0            0.961429   

                                annual_plant_shift_mw  annual_fuel_ratio  \
plant_id_eia subplant_id month     

In [19]:
numeric_2020.head()

hours_in_month_subplant  gross_generation_mwh  \
plant_id_eia subplant_id month                                                  
3            0           1                        744.0                3311.0   
                         2                        696.0                2375.0   
             1           1                        744.0                1397.0   
             2           1                        744.0                   0.0   
                         4                        720.0               61442.0   

                                minimum_gross_generation_mwh  \
plant_id_eia subplant_id month                                 
3            0           1                               0.0   
                         2                               0.0   
             1           1                               0.0   
             2           1                               0.0   
                         4                               0.0   

                                maximum_gross_generation_mwh  capacity_mw  \
plant_id_eia subplant_id month                                              
3            0           1                             29.70        153.1   
                         2                             29.00        153.1   
             1           1                             20.00        153.1   
             2           1                              0.00        403.7   
                         4                            358.24        403.7   

                                net_generation_mwh  monthly_subplant_ratio  \
plant_id_eia subplant_id month                                               
3            0           1                     NaN                     NaN   
                         2                     0.0                0.000000   
             1           1                     NaN                     NaN   
             2           1                     NaN                     NaN   
                         4                 57613.8                0.937694   

                                monthly_subplant_shift_mw  \
plant_id_eia subplant_id month                              
3            0           1                            NaN   
                         2                      -3.412356   
             1           1                            NaN   
             2           1                            NaN   
                         4                      -5.316944   

                                annual_subplant_ratio  \
plant_id_eia subplant_id month                          
3            0           1                   0.000000   
                         2                   0.000000   
             1           1                        NaN   
             2           1                   0.920113   
                         4                   0.920113   

                                annual_subplant_shift_mw  ...  \
plant_id_eia subplant_id month                            ...   
3            0           1                     -3.412356  ...   
                         2                     -3.412356  ...   
             1           1                           NaN  ...   
             2           1                     -3.650425  ...   
                         4                     -3.650425  ...   

                                hours_in_month_plant  annual_plant_ratio  \
plant_id_eia subplant_id month                                             
3            0           1                   28465.0            0.966946   
                         2                   28465.0            0.966946   
             1           1                   28465.0            0.966946   
             2           1                   28465.0            0.966946   
                         4                   28465.0            0.966946   

                                annual_plant_shift_mw  annual_fuel_ratio  \
plant_id_eia subplant_id month     

# Annual data

This is the highest priority for triage and the easiest 

In [102]:
y1 = 2020
y2 = 2021
dat_y1 = pd.read_csv(f"../../data/results/{y1}/plant_data/annual/us_units/plant_data.csv")
dat_y2 = pd.read_csv(f"../../data/results/{y2}/plant_data/annual/us_units/plant_data.csv")


In [103]:
file_type_to_index_cols = {
    "plant_data":["plant_id_eia"]
}

In [104]:
dat_y1 = dat_y1.set_index(file_type_to_index_cols['plant_data'])
dat_y2 = dat_y2.set_index(file_type_to_index_cols['plant_data'])

In [105]:
emission_cols = [c for c in dat_y1.columns if "mass_lb" in c]

In [106]:
in_both = dat_y1.index.intersection(dat_y2.index)
dat_y1 = dat_y1.loc[in_both]
dat_y2 = dat_y2.loc[in_both]


In [107]:
non_zero_gen = dat_y1[(dat_y1.net_generation_mwh > 0)].index.intersection(dat_y2[(dat_y2.net_generation_mwh > 0)].index)
print(len(non_zero_gen))
dat_y1 = dat_y1.loc[non_zero_gen]
dat_y2 = dat_y2.loc[non_zero_gen]

9619


In [108]:
rates_y1 = dat_y1[emission_cols].div(dat_y1.net_generation_mwh, axis=0)
rates_y1.columns = [c+"_per_mwh" for c in rates_y1.columns]

rates_y2 = dat_y2[emission_cols].div(dat_y2.net_generation_mwh, axis=0)
rates_y2.columns = [c+"_per_mwh" for c in rates_y2.columns]

In [109]:
# Drop emission rate zero plants. 
# TODO: ID rates zero in one year and not in other
non_zero_rates = rates_y1.loc[(rates_y1 != 0).all(axis=1)].index.intersection(rates_y2.loc[(rates_y2 != 0).all(axis=1)].index)
rates_y1 = rates_y1.loc[non_zero_rates]
rates_y2 = rates_y2.loc[non_zero_rates]

In [110]:
frac_diff = abs(rates_y2 - rates_y1)/rates_y1


In [111]:
px.line(frac_diff.quantile(np.arange(0, 1, .01)))

In [112]:
# Top 5% of differences: 
frac_diff.quantile(.95) * 100

co2_mass_lb_per_mwh                               60.863671
ch4_mass_lb_per_mwh                               66.498564
n2o_mass_lb_per_mwh                               78.314784
co2e_mass_lb_per_mwh                              61.121130
nox_mass_lb_per_mwh                               83.200933
so2_mass_lb_per_mwh                              134.481142
co2_mass_lb_for_electricity_per_mwh               51.173389
ch4_mass_lb_for_electricity_per_mwh               59.801308
n2o_mass_lb_for_electricity_per_mwh               71.082721
co2e_mass_lb_for_electricity_per_mwh              51.194756
nox_mass_lb_for_electricity_per_mwh               76.559519
so2_mass_lb_for_electricity_per_mwh              116.823586
co2_mass_lb_adjusted_per_mwh                      67.334545
ch4_mass_lb_adjusted_per_mwh                      66.913412
n2o_mass_lb_adjusted_per_mwh                      78.590904
co2e_mass_lb_adjusted_per_mwh                     65.233858
nox_mass_lb_adjusted_per_mwh            

In [114]:
# What fraction of differences are above 10%? 
((frac_diff > 1).sum(axis=0)/len(frac_diff)).sort_values(ascending=False) * 100

so2_mass_lb_adjusted_per_mwh                     5.826051
so2_mass_lb_per_mwh                              5.784436
so2_mass_lb_for_electricity_adjusted_per_mwh     5.534748
so2_mass_lb_for_electricity_per_mwh              5.493134
n2o_mass_lb_adjusted_per_mwh                     3.370787
n2o_mass_lb_per_mwh                              3.329172
n2o_mass_lb_for_electricity_adjusted_per_mwh     2.996255
n2o_mass_lb_for_electricity_per_mwh              2.954640
nox_mass_lb_per_mwh                              2.829796
nox_mass_lb_adjusted_per_mwh                     2.829796
nox_mass_lb_for_electricity_adjusted_per_mwh     2.496879
nox_mass_lb_for_electricity_per_mwh              2.496879
ch4_mass_lb_adjusted_per_mwh                     2.413650
co2_mass_lb_adjusted_per_mwh                     2.372035
ch4_mass_lb_per_mwh                              2.372035
co2e_mass_lb_adjusted_per_mwh                    2.122347
ch4_mass_lb_for_electricity_adjusted_per_mwh     1.955888
ch4_mass_lb_fo

In [ ]:
# TODO: break down by plant capacity 

# BA-level, annual 

(to come: BA level, monthly)

In [123]:
y1 = 2019
y2 = 2020

path1 = results_folder(f"{y1}/power_sector_data/annual/us_units/")
path2 = results_folder(f"{y2}/power_sector_data/annual/us_units/")

In [135]:
ba_list = {d.replace(".csv","") for d in os.listdir(path1)}
ba_list2 = {d.replace(".csv","") for d in os.listdir(path2)}
bas_to_check = set.intersection(ba_list, ba_list2)
bas_cant_check = set.difference(set.union(ba_list2, ba_list), bas_to_check)

In [136]:
# Warning: can't check these BAs since they only exist in one or the other year.
bas_cant_check

{'AZMS'}

In [185]:
one_year_fuels.values

array([], dtype=object)

In [232]:
all_ba_changes = [] 
all_bad_rates = []
for ba in bas_to_check: 
    ba_y1 = pd.read_csv(f"{path1}/{ba}.csv", dtype=get_dtypes(), index_col="fuel_category")
    ba_y2 = pd.read_csv(f"{path2}/{ba}.csv", dtype=get_dtypes(), index_col="fuel_category")
        # Identify any differences in fuel types between years
    any_year_fuels = ba_y1.index.union(ba_y2.index)
    both_year_fuels = ba_y1.index.intersection(ba_y2.index)
    one_year_fuels = any_year_fuels.difference(both_year_fuels)
    if len(one_year_fuels) > 0: 
        print(f"{ba} Warning: fuels {one_year_fuels.values} in only one of comparison years")

    # Is this BA all zero? 
    # TODO: separate check by year, expand to check for all zero/nan (eg, RIMS 2020)
    if (((ba_y1 == 0) | ba_y1.isna()).all().all()):
        print(f"Warning: ba {ba} {y1} is all zero/NaN.")
        
    if (((ba_y2 == 0) | ba_y2.isna()).all().all()):
        print(f"Warning: ba {ba} {y2} is all zero/NaN.")
    
    rate_cols = [c for c in ba_y1.columns if "rate" in c]
    diff = (ba_y1[rate_cols] - ba_y2[rate_cols])
    frac_change = diff/ba_y1[rate_cols]
    
    # Renewables are often zero in both years -- don't want NaN here 
    frac_change[diff == 0] = 0 

    # But if there was zero emissions in y1 and non-zero in y2, we want inf not NaN
    if frac_change.isna().any().any(): 
        print(f"{ba} Warning! some non-zero in y2 but zero in y1")
        frac_change[frac_change.isna()] = np.inf

    ba_changes = frac_change.melt(ignore_index=False, value_name="frac_change").reset_index()
    ba_changes["BA"] = ba
    ba_changes = ba_changes.set_index(["BA","fuel_category","variable"])

    diff_size = diff.melt(ignore_index=False, value_name="diff").reset_index()
    diff_size["BA"] = ba
    diff_size = diff_size.set_index(["BA","fuel_category","variable"])

    ba_changes["difference"] = diff_size["diff"]
    
    # Find and exclude differences from rates which are NaN in one or other year 
    bad_rates = diff.isna().melt(ignore_index=False, value_name="is_na").reset_index()
    bad_rates["BA"] = ba
    bad_rates = bad_rates.set_index(["BA","fuel_category","variable"])

    all_bad_rates.append(bad_rates[bad_rates.is_na])
    ba_changes = ba_changes.loc[bad_rates[bad_rates.is_na == False].index]
    
    all_ba_changes.append(ba_changes)


AMPL Warning: fuels ['hydro' 'natural_gas'] in only one of comparison years
AMPL Warning! some non-zero in y2 but zero in y1
GVL Warning: fuels ['coal'] in only one of comparison years
GVL Warning! some non-zero in y2 but zero in y1
AVRN Warning: fuels ['natural_gas'] in only one of comparison years
AVRN Warning! some non-zero in y2 but zero in y1
WALC Warning: fuels ['wind'] in only one of comparison years
WALC Warning! some non-zero in y2 but zero in y1
HST Warning: fuels ['natural_gas' 'petroleum'] in only one of comparison years
HST Warning! some non-zero in y2 but zero in y1
NBSO Warning: fuels ['petroleum'] in only one of comparison years
NBSO Warning! some non-zero in y2 but zero in y1
AZPS Warning! some non-zero in y2 but zero in y1
BANC Warning: fuels ['petroleum'] in only one of comparison years
BANC Warning! some non-zero in y2 but zero in y1
FMPP Warning: fuels ['biomass'] in only one of comparison years
FMPP Warning! some non-zero in y2 but zero in y1
FPC Warning: fuels ['

In [244]:
to_export = pd.concat(all_ba_changes)
to_export["abs_diff"] = to_export.difference.abs()
to_export = to_export.sort_values("abs_diff", ascending=False)
to_export = to_export.drop(columns="abs_diff")

In [245]:
os.makedirs(results_folder("validation"), exist_ok=True)
to_export.to_csv(f"{results_folder('validation')}/yoy_compare_{y1}_{y2}")

In [226]:
# Find NaN rates 
bad_rates = diff.isna().melt(ignore_index=False, value_name="is_na").reset_index()
bad_rates["BA"] = ba
bad_rates = bad_rates.set_index(["BA","fuel_category","variable"])

#bad_rates = bad_rates[bad_rates.is_na]

In [228]:
bad_rates[bad_rates.is_na == False].index

MultiIndex([('RIMS', 'petroleum', ...),
            ('RIMS',     'total', ...),
            ('RIMS', 'petroleum', ...),
            ('RIMS',     'total', ...),
            ('RIMS', 'petroleum', ...),
            ('RIMS',     'total', ...),
            ('RIMS', 'petroleum', ...),
            ('RIMS',     'total', ...)],
           names=['BA', 'fuel_category', 'variable'])

In [230]:
ba_changes.loc[bad_rates[bad_rates.is_na == False].index]

frac_change  \
BA   fuel_category variable                                                          
RIMS petroleum     generated_ch4_rate_lb_per_mwh_for_electricity               1.0   
     total         generated_ch4_rate_lb_per_mwh_for_electricity               1.0   
     petroleum     generated_n2o_rate_lb_per_mwh_for_electricity               0.0   
     total         generated_n2o_rate_lb_per_mwh_for_electricity               0.0   
     petroleum     generated_ch4_rate_lb_per_mwh_for_electricity_a...          1.0   
     total         generated_ch4_rate_lb_per_mwh_for_electricity_a...          1.0   
     petroleum     generated_n2o_rate_lb_per_mwh_for_electricity_a...          0.0   
     total         generated_n2o_rate_lb_per_mwh_for_electricity_a...          0.0   

                                                                       difference  
BA   fuel_category variable                                                        
RIMS petroleum     generated_ch4_rate_lb_per_mwh_for_electricity           0.0906  
     total         generated_ch4_rate_lb_per_mwh_for_electricity           0.0906  
     petroleum     generated_n2o_rate_lb_per_mwh_for_electricity           0.0000  
     total         generated_n2o_rate_lb_per_mwh_for_electricity           0.0000  
     petroleum     generated_ch4_rate_lb_per_mwh_for_electricity_a...      0.0906  
     total         generated_ch4_rate_lb_per_mwh_for_electricity_a...      0.0906  
     petroleum     generated_n2o_rate_lb_per_mwh_for_electricity_a...      0.0000  
     total         generated_n2o_rate_lb_per_mwh_for_electricity_a...      0.0000